# Interpretability of neural networks


## Model Explanation
Activiation Maximization : Finding a prototype of each target class.

**TODO** : add more details and example

## Explaining DNN Decisions
**NOTATIONS**

$R_i(\boldsymbol{x})$ = relevance score of pixel $i$ of an image $\boldsymbol{x}$

$f(\boldsymbol{x})$ = unnormalized prediction score ( aka. activation values before softmax layer )


### Sensitivity Analysis
Computing rate of change of output respect to input variables.

```
What makes an image more/less a car?
```
$$
R_i(\boldsymbol{x}) = \frac{\partial f(\boldsymbol{x})}{\partial x_i}
$$
Commonly people use :
$$
R_i(\boldsymbol{x}) = \Big ( \frac{\partial f(\boldsymbol{x})}{\partial x_i} \Big )^2
$$
DOUBT
- Why is it better to use $(..)^2$?
- Does Saliency map use this method?

### Simple Taylor Decomposion
Decomposing $f(\boldsymbol{x})$ as sum of $R_i(\boldsymbol{x})$
$$
f(\boldsymbol{x}) = \sum_{i=1}^{d}{ R_i(\boldsymbol{x}) }  + O(\boldsymbol{x}\boldsymbol{x}^T)
$$
where
$$
R_i(\boldsymbol{x}) = \frac{\partial f(\boldsymbol{x})}{\partial x_i} 
\Bigg |_{\boldsymbol{x} = \boldsymbol{\widetilde{x}}} \cdot ( x_i - \widetilde{x}_i )
$$
DOUBT
- Is $x_i - \widetilde{x}_i$ a vector or scalar?

#### Derivation of Simple Taylor Decomposion
\begin{align}
f(\boldsymbol{x})  &= f(\boldsymbol{\widetilde{x}}) + \nabla_{\boldsymbol{x}} f(\boldsymbol{x}) 
\big |_{\boldsymbol{x} = \boldsymbol{\widetilde{x}}} \cdot ( \boldsymbol{x} - \boldsymbol{ \widetilde{x} } ) + O(\nabla_{\boldsymbol{x}}^{2} ) + O(\nabla_{\boldsymbol{x}}^{3} ) + ...
\\
&= f(\boldsymbol{\widetilde{x}}) + \sum_{i=1}^{d}{\frac{\partial f(\boldsymbol{x})}{\partial x_i}} \Big |_{x_i = \widetilde{x}_i}  ( x_i - \widetilde{x}_i ) + O(\boldsymbol{x}\boldsymbol{x}^T) \\
&= \sum_{i=1}^{d}{\frac{\partial f(\boldsymbol{x})}{\partial x_i}} \Big |_{x_i = \widetilde{x}_i}  ( x_i - \widetilde{x}_i ) +
O(\boldsymbol{x}\boldsymbol{x}^T)
\tag*{(choose  $\boldsymbol{\widetilde{x}}  \to f(\boldsymbol{\widetilde{x}}) = 0 $  )}
\end{align}

#### ReLU Network without biases
For networks that based on **ReLU** activator: $\textrm{RL}(x) = max(0, x)$,  we can omit $O(\boldsymbol{x}\boldsymbol{x}^T)$ because $$\forall i \ge 2 : \nabla_{\boldsymbol{x}}^{i}f(\boldsymbol{x}) = 0 $$
and $\boldsymbol{\widetilde{x}}$ can be found by :
$$
\boldsymbol{\widetilde{x}} = \lim_{\epsilon \to 0} \epsilon \boldsymbol{x} \tag*{DOUBT : what does this mean?}
$$

$$
\lim_{\epsilon \to 0} f(\epsilon \boldsymbol{x}) = \lim_{\epsilon \to 0} \epsilon f(\boldsymbol{x})
$$

As a result, 
$$
f(\boldsymbol{x}) = \sum_{i=1}^{d} R_i({\boldsymbol{x}})
$$
and the relevance score is :
$$
R_i(\boldsymbol{x}) = \frac{\partial f(\boldsymbol{x}) }{\partial x_i } x_i \tag*{DOUBT : how?}
$$ 
where $ \frac{\partial f(\boldsymbol{x}) }{\partial x_i } $ is sensivity and $x_i$ is saliency of input value


Deriavation of $R_i(\boldsymbol{x})$

\begin{align}
f'(\widetilde{x}) &= \lim_{ h \to 0} \frac{ f( \widetilde{x}+ h ) - f(\widetilde{x})}{h} \\ 
&= \lim_{ h \to 0} \frac{ f( cx+ h ) - f(cx)}{h} \tag*{($\widetilde{x} = \lim_{\epsilon \to 0} \epsilon x = cx$)} \\
&= \lim_{ h \to 0} \frac{ cf(x+ h/c) - cf(x)}{h} \tag*{(ReLU property)} \\
&= c\lim_{ h \to 0} \frac{ f(x+ h/c) - f(x)}{h} \\
&= c\lim_{ h \to 0} \frac{ f(x+ h/c) - f(x)}{h} \cdot \frac{c}{c} \\
&= c\lim_{ h \to 0} \frac{ f(x+ h/c) - f(x)}{h/c} \cdot \frac{1}{c} \\
&= f'(x)
\end{align}
Thus,

\begin{align}
\frac{\partial f(\boldsymbol{x})}{\partial x_i} 
\Bigg |_{\boldsymbol{x} = \boldsymbol{\widetilde{x}}}  = \frac{\partial f(\boldsymbol{x})}{\partial x_i}
\end{align}


### Relevance Propagation
- Exploit feed-forward graph of the model. Redistributing prediction score back until input pixels.
- Redistribution rule must satisfy **relevance conservation** priniciple ( aka no enery loss during the process )

**NOTATION**
- $R_{j \leftarrow k }$ is relevance score propagated from neuron $k$ to $j$ where $k$ in layer **i-th** and $j$ is in **(i-1)-th**

To satisfy  **relevance conservation**, we need :

- $\sum_{j} R_{j \leftarrow k} = R_k$
- $R_j = \sum_{k} R_{j \leftarrow k}$

Putting these constraints together we have :
\begin{align}
f(x) &= \sum_{k} R_{k} 
= \sum_{k} \sum_{j} R_{j \leftarrow k} 
= \sum_{j} \sum_{k} R_{j \leftarrow k} 
= \sum_{j} R_{j} 
= \cdots  = \sum_{i=1}^{d} R_i
\end{align}

Advantages of decomposition method:
- DOUBT : if the number of input variables  is limited, the analysis can be represented as a pie chart, histogram
- Pooling. For example, relevance scores of RBG channels can be aggregated and represented as the score of that pixel.
- Filtering

#### Layer-wise Relevance Propagation

Other methods that exploit network structure but don't use decomposition:
- TODO: deconvolution 
- TODO: guided back-prop

# Next steps